In [1]:
# Install required packages for options trading and volatility modeling
# Run this cell first before running any other code

# Fix dependency conflicts by installing packages in specific order
print("Installing core packages...")
!pip install --upgrade pip

# Install core scientific packages first
!pip install pandas numpy scipy matplotlib seaborn plotly

# Install financial packages with conflict resolution
print("\nInstalling financial packages...")
!pip install scikit-learn statsmodels arch

# Install alpaca-trade-api with relaxed dependencies to avoid websockets conflict
!pip install alpaca-trade-api --no-deps
!pip install requests urllib3 websocket-client aiohttp PyYAML deprecation msgpack

# Install remaining packages
!pip install yfinance jupyter ipywidgets tqdm python-dateutil pytz

# Skip ta-lib for now (requires additional system dependencies)
print("\nNote: ta-lib installation skipped (requires system dependencies)")
print("If needed later, install with: conda install -c conda-forge ta-lib")

# Improved verification that handles dependency conflicts
print("\n" + "="*50)
print("PACKAGE VERIFICATION")
print("="*50)

import importlib
import sys

packages_to_check = {
    'alpaca_trade_api': 'alpaca-trade-api',
    'pandas': 'pandas',
    'numpy': 'numpy',
    'scipy': 'scipy',
    'matplotlib': 'matplotlib',
    'seaborn': 'seaborn',
    'plotly': 'plotly',
    'sklearn': 'scikit-learn',
    'statsmodels': 'statsmodels',
    'arch': 'arch',
    'yfinance': 'yfinance',
    'tqdm': 'tqdm',
    'requests': 'requests',
    'dateutil': 'python-dateutil',
    'pytz': 'pytz'
}

successful_imports = 0
total_packages = len(packages_to_check)

for import_name, package_name in packages_to_check.items():
    try:
        module = importlib.import_module(import_name)
        version = getattr(module, '__version__', 'unknown')
        print(f"✓ {package_name:<20} {version}")
        successful_imports += 1
    except ImportError as e:
        print(f"✗ {package_name:<20} IMPORT ERROR: {str(e)}")
    except Exception as e:
        print(f"? {package_name:<20} UNKNOWN ERROR: {str(e)}")

print(f"\nSUMMARY: {successful_imports}/{total_packages} packages successfully imported")
print(f"Python version: {sys.version}")

if successful_imports == total_packages:
    print("\n🎉 All packages installed successfully!")
    print("You can now proceed with the Alpaca data fetching code.")
else:
    print(f"\n⚠️  {total_packages - successful_imports} package(s) had issues.")
    print("Try running individual install commands for failed packages.")

# Test Alpaca API import specifically
print("\n" + "-"*30)
print("ALPACA API TEST")
print("-"*30)
try:
    import alpaca_trade_api as tradeapi
    print("✓ Alpaca Trade API imported successfully")
    print(f"✓ Version: {tradeapi.__version__}")
    print("✓ Ready for data fetching!")
except Exception as e:
    print(f"✗ Alpaca import failed: {e}")
    print("Try: !pip install --force-reinstall alpaca-trade-api")

Installing core packages...
  Using cached pip-25.1.1-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-25.1.1-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:
      Successfully uninstalled pip-24.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 8.5 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [plotly]2m1/2 [plotly]

Installing financial packages...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10/10 [aiohttp]9/10 [aiohttp]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
alpaca-trade-api 3.2.0 requires msgpack==1.0.3, but you have msgpack 1.1.1 which is incompatible.
alpaca-trade-api 3.2.0 requires PyYAML==6.0.1, but you have pyyaml 6.0.2 which is incompatible.
alpaca-trade-api 3.2.0 requires urllib3<2,>1.24, but you have urllib3 2.3.0 which is incom

In [2]:
import alpaca_trade_api as tradeapi
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pytz
from typing import Optional, List, Dict
import time
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class AlpacaIntradayDataFetcher:
    """
    Fetches historical intraday data from Alpaca API for volatility modeling.
    Handles rate limiting, data cleaning, and formats data for volatility calculations.
    """

    def __init__(self, api_key: str, secret_key: str, base_url: str = "https://paper-api.alpaca.markets"):
        """
        Initialize Alpaca API connection.

        Args:
            api_key: Alpaca API key
            secret_key: Alpaca secret key
            base_url: Alpaca base URL (paper or live)
        """
        self.api = tradeapi.REST(api_key, secret_key, base_url, api_version='v2')
        self.market_tz = pytz.timezone('America/New_York')

    def get_intraday_bars(self,
                         symbol: str,
                         start_date: datetime,
                         end_date: datetime,
                         timeframe: str = '1Min',
                         adjustment: str = 'split') -> pd.DataFrame:
        """
        Fetch intraday bars for a symbol over a date range.

        Args:
            symbol: Stock symbol (e.g., 'SPY', 'AAPL')
            start_date: Start date for data
            end_date: End date for data
            timeframe: Bar timeframe ('1Min', '5Min', '15Min', '1Hour')
            adjustment: Price adjustment type ('raw', 'split', 'dividend', 'all')

        Returns:
            DataFrame with OHLCV data and timestamp index
        """
        try:
            # Convert timeframe to Alpaca format
            timeframe_map = {
                '1Min': tradeapi.TimeFrame.Minute,
                '5Min': tradeapi.TimeFrame(5, tradeapi.TimeFrameUnit.Minute),
                '15Min': tradeapi.TimeFrame(15, tradeapi.TimeFrameUnit.Minute),
                '1Hour': tradeapi.TimeFrame.Hour,
                '1Day': tradeapi.TimeFrame.Day
            }

            tf = timeframe_map.get(timeframe, tradeapi.TimeFrame.Minute)

            logger.info(f"Fetching {timeframe} bars for {symbol} from {start_date} to {end_date}")

            # Fetch bars with rate limiting
            bars = self.api.get_bars(
                symbol,
                tf,
                start=start_date.isoformat(),
                end=end_date.isoformat(),
                adjustment=adjustment,
                limit=10000  # Max per request
            ).df

            if bars.empty:
                logger.warning(f"No data returned for {symbol}")
                return pd.DataFrame()

            # Clean and format data
            bars = self._clean_bars_data(bars, symbol)

            logger.info(f"Retrieved {len(bars)} bars for {symbol}")
            return bars

        except Exception as e:
            logger.error(f"Error fetching data for {symbol}: {str(e)}")
            raise

    def get_multiple_symbols_data(self,
                                 symbols: List[str],
                                 start_date: datetime,
                                 end_date: datetime,
                                 timeframe: str = '1Min',
                                 delay_between_requests: float = 0.1) -> Dict[str, pd.DataFrame]:
        """
        Fetch intraday data for multiple symbols with rate limiting.

        Args:
            symbols: List of stock symbols
            start_date: Start date for data
            end_date: End date for data
            timeframe: Bar timeframe
            delay_between_requests: Delay between API calls (seconds)

        Returns:
            Dictionary mapping symbols to their DataFrames
        """
        data = {}

        for symbol in symbols:
            try:
                df = self.get_intraday_bars(symbol, start_date, end_date, timeframe)
                if not df.empty:
                    data[symbol] = df

                # Rate limiting
                time.sleep(delay_between_requests)

            except Exception as e:
                logger.error(f"Failed to fetch data for {symbol}: {str(e)}")
                continue

        return data

    def _clean_bars_data(self, bars: pd.DataFrame, symbol: str) -> pd.DataFrame:
        """
        Clean and format bars data for volatility calculations.

        Args:
            bars: Raw bars DataFrame from Alpaca
            symbol: Stock symbol for reference

        Returns:
            Cleaned DataFrame
        """
        # Remove any rows with missing data
        bars = bars.dropna()

        # Ensure timestamp is timezone-aware and convert to market timezone
        if bars.index.tz is None:
            bars.index = bars.index.tz_localize('UTC')
        bars.index = bars.index.tz_convert(self.market_tz)

        # Add symbol column
        bars['symbol'] = symbol

        # Calculate returns for volatility computation
        bars['log_return'] = np.log(bars['close'] / bars['close'].shift(1))
        bars['simple_return'] = bars['close'].pct_change()

        # Filter out extreme outliers (likely data errors)
        # Remove returns > 50% in a single bar (adjust threshold as needed)
        bars = bars[abs(bars['log_return']) < 0.5]

        # Add trading session indicators
        bars['hour'] = bars.index.hour
        bars['minute'] = bars.index.minute
        bars['is_market_hours'] = (
            (bars['hour'] >= 9) &
            ((bars['hour'] < 16) | ((bars['hour'] == 9) & (bars['minute'] >= 30)))
        )

        return bars

    def calculate_daily_realized_volatility(self,
                                          bars_df: pd.DataFrame,
                                          method: str = 'close_to_close',
                                          annualize: bool = True) -> pd.DataFrame:
        """
        Calculate daily realized volatility from intraday data.

        Args:
            bars_df: DataFrame with intraday bars
            method: Method for RV calculation ('close_to_close', 'high_low', 'rogers_satchell')
            annualize: Whether to annualize the volatility

        Returns:
            DataFrame with daily realized volatility
        """
        if bars_df.empty:
            return pd.DataFrame()

        # Group by date
        daily_data = []

        for date, day_data in bars_df.groupby(bars_df.index.date):
            # Filter to market hours only
            market_data = day_data[day_data['is_market_hours']]

            if len(market_data) < 10:  # Need minimum bars for reliable RV
                continue

            if method == 'close_to_close':
                # Sum of squared log returns
                rv = (market_data['log_return'] ** 2).sum()

            elif method == 'high_low':
                # Parkinson estimator using high-low
                hl_data = market_data.groupby(market_data.index.date).agg({
                    'high': 'max',
                    'low': 'min',
                    'open': 'first',
                    'close': 'last'
                })
                rv = (np.log(hl_data['high'] / hl_data['low']) ** 2).iloc[0]

            elif method == 'rogers_satchell':
                # Rogers-Satchell estimator (drift-independent)
                rs_terms = (
                    np.log(market_data['high'] / market_data['close']) *
                    np.log(market_data['high'] / market_data['open']) +
                    np.log(market_data['low'] / market_data['close']) *
                    np.log(market_data['low'] / market_data['open'])
                )
                rv = rs_terms.sum()

            # Annualize if requested (252 trading days)
            if annualize:
                rv = rv * 252

            daily_data.append({
                'date': date,
                'realized_vol': np.sqrt(rv),
                'realized_var': rv,
                'n_bars': len(market_data)
            })

        return pd.DataFrame(daily_data).set_index('date')

    def get_data_for_volatility_modeling(self,
                                       symbol: str,
                                       lookback_days: int = 252,
                                       timeframe: str = '1Min') -> Dict:
        """
        Get complete dataset for volatility modeling (HAR-RV).

        Args:
            symbol: Stock symbol
            lookback_days: Number of days of historical data
            timeframe: Intraday timeframe

        Returns:
            Dictionary with intraday bars and daily realized volatility
        """
        # Calculate date range
        end_date = datetime.now(self.market_tz)
        start_date = end_date - timedelta(days=int(lookback_days * 1.5))  # Buffer for weekends/holidays

        # Get intraday data
        bars = self.get_intraday_bars(symbol, start_date, end_date, timeframe)

        if bars.empty:
            return {'bars': pd.DataFrame(), 'daily_rv': pd.DataFrame()}

        # Calculate daily realized volatility
        daily_rv = self.calculate_daily_realized_volatility(bars)

        # Keep only the requested number of days
        daily_rv = daily_rv.tail(lookback_days)

        logger.info(f"Prepared {len(daily_rv)} days of RV data for {symbol}")

        return {
            'bars': bars,
            'daily_rv': daily_rv,
            'symbol': symbol,
            'timeframe': timeframe,
            'data_range': (daily_rv.index.min(), daily_rv.index.max()) if not daily_rv.empty else None
        }


# Example usage and testing
if __name__ == "__main__":
    # Example usage - replace with your actual API keys
    API_KEY = "6a3ea255-7f31-455c-8e41-6e444b1c4fc6"
    SECRET_KEY = "ig5CGnl3c1jXEepU6VK5DPXgsV5WSOBYrIJGk70T"

    # Initialize fetcher
    fetcher = AlpacaIntradayDataFetcher(API_KEY, SECRET_KEY)

    # Example 1: Get recent SPY data for volatility modeling
    spy_data = fetcher.get_data_for_volatility_modeling('SPY', lookback_days=60)

    if not spy_data['daily_rv'].empty:
        print(f"SPY Realized Volatility Summary:")
        print(spy_data['daily_rv']['realized_vol'].describe())
        print(f"\nLatest RV: {spy_data['daily_rv']['realized_vol'].iloc[-1]:.4f}")

    # Example 2: Get data for multiple symbols
    symbols = ['SPY', 'QQQ', 'IWM']
    end_date = datetime.now()
    start_date = end_date - timedelta(days=5)

    multi_data = fetcher.get_multiple_symbols_data(
        symbols, start_date, end_date, timeframe='5Min'
    )

    for symbol, df in multi_data.items():
        print(f"\n{symbol}: {len(df)} bars retrieved")
        if not df.empty:
            print(f"Date range: {df.index.min()} to {df.index.max()}")

INFO:__main__:Fetching 1Min bars for SPY from 2025-03-29 19:16:12.225279-04:00 to 2025-06-27 19:16:12.225279-04:00
ERROR:__main__:Error fetching data for SPY: 403 Client Error: Forbidden for url: https://data.alpaca.markets/v2/stocks/SPY/bars?timeframe=1Min&adjustment=split&start=2025-03-29T19%3A16%3A12.225279-04%3A00&end=2025-06-27T19%3A16%3A12.225279-04%3A00&limit=10000


HTTPError: 403 Client Error: Forbidden for url: https://data.alpaca.markets/v2/stocks/SPY/bars?timeframe=1Min&adjustment=split&start=2025-03-29T19%3A16%3A12.225279-04%3A00&end=2025-06-27T19%3A16%3A12.225279-04%3A00&limit=10000